In [1]:
import os
import pandas as pd
import nltk
nltk.download('wordnet')
wn = nltk.corpus.wordnet
from digital_manuscript import BnF

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\danac\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
path = os.getcwd() + '/../manuscript-object/thesaurus/animal.csv'

df = pd.read_csv(path)
terms = list(set(df['prefLabel_en']))
simple_terms = [t for t in terms if ' ' not in t]
simple_terms

['petit-gri',
 'squirrel',
 'cat',
 'turtle',
 'daot',
 'ortolan',
 'mealworm',
 'turtledofe',
 'mutton',
 'crab',
 'weasel',
 'earthworm',
 'duck',
 'silkworm',
 'codfish',
 'spider',
 'louse',
 'passerine',
 'cicada',
 'fish',
 'mussel',
 'water-dog',
 'bittern',
 'pig',
 'oyster',
 'lamb',
 'viper',
 'nightingale',
 'beef',
 'snake',
 'sheep',
 'pigeon',
 'hog',
 'cow',
 'eagle',
 'colt',
 'lizard',
 'snail',
 'wolf',
 'goldfinch',
 'rabbit',
 'mule',
 'porcupine',
 'partridge',
 'lark',
 'toad',
 'ant',
 'horse',
 'dogfish',
 'calandra',
 'frog',
 'barbel',
 'grasshopper',
 'cuckoo',
 'flea',
 'hen',
 'fly',
 'dog',
 'rat',
 'og',
 'shell',
 'swine',
 'calendra',
 'hart',
 'calf',
 'butterfly',
 'ox',
 'mouse',
 'crayfish',
 'aucupio',
 'verdaule',
 'bombicum',
 'siskin',
 'bird',
 'chaffinch',
 'pork',
 'dragon',
 'chicken',
 'chick',
 'dormouse',
 'worm',
 'cuttlefish',
 'linnet',
 'tellin',
 'swallow',
 'bat',
 'swan',
 'goat',
 'ewe',
 'crow',
 'animal',
 'human']

In [3]:
term = 'dog'
dog = wn.synsets(term, pos=wn.NOUN)
dog
# hyper = dog[0].hypernyms()
# for x in hyper:
#     print(x, x.hypernyms())

# from nltk.corpus import wordnet as wn
# for ss in wn.synsets(term):
#     print (ss, ss.hypernyms())

[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01')]

In [4]:
animal = wn.synset('animal.n.01')

class r_node:
    def __init__(self, lemma, children=[]):
        self.lemma = lemma
        
#         if lemma == animal:
#             print(children + [lemma,])
        
        self.children = children
        self.hypernyms = lemma.hypernyms()
        self.parents = [r_node(h, children + [lemma,]) for h in self.hypernyms]
        
    def return_animal_path(self):
        if self.lemma == animal:
            return [[self.lemma,],]
        else:
            parent_paths = [n.return_animal_path() for n in self.parents]
            parent_paths = [p for p in parent_paths if len(p) > 0]
            paths = []
            for p in parent_paths:
                paths += p
            for p in paths:
                p.append(self.lemma)
            return paths

In [5]:
dog = wn.synsets('dog', pos=wn.NOUN)[0]
dn = r_node(dog)
dn.return_animal_path()
# print([p.parents[0].lemma for p in dn.parents])

# animal


[[Synset('animal.n.01'),
  Synset('chordate.n.01'),
  Synset('vertebrate.n.01'),
  Synset('mammal.n.01'),
  Synset('placental.n.01'),
  Synset('carnivore.n.01'),
  Synset('canine.n.02'),
  Synset('dog.n.01')],
 [Synset('animal.n.01'), Synset('domestic_animal.n.01'), Synset('dog.n.01')]]

In [6]:
squirrel = wn.synsets('squirrel')[0]
sn = r_node(squirrel)
sn.return_animal_path()

[[Synset('animal.n.01'),
  Synset('chordate.n.01'),
  Synset('vertebrate.n.01'),
  Synset('mammal.n.01'),
  Synset('placental.n.01'),
  Synset('rodent.n.01'),
  Synset('squirrel.n.01')]]

In [7]:
fish = wn.synsets('fish', pos=wn.NOUN)[0]
fn = r_node(fish)
fn.return_animal_path()

[[Synset('animal.n.01'),
  Synset('chordate.n.01'),
  Synset('vertebrate.n.01'),
  Synset('aquatic_vertebrate.n.01'),
  Synset('fish.n.01')]]

In [8]:
louse = wn.synsets('louse')[0]
ln = r_node(louse)
ln.return_animal_path()

[[Synset('animal.n.01'),
  Synset('invertebrate.n.01'),
  Synset('arthropod.n.01'),
  Synset('insect.n.01'),
  Synset('louse.n.01')]]

In [9]:
oyster = wn.synsets('oyster')[0]
on = r_node(oyster)
on.return_animal_path()

[[Synset('animal.n.01'),
  Synset('invertebrate.n.01'),
  Synset('mollusk.n.01'),
  Synset('bivalve.n.01'),
  Synset('oyster.n.01')]]

In [10]:
def get_animal_paths(lemma):
    paths = []
    node = r_node(lemma)
    
    
    
    

In [11]:
squirrel = wn.synsets('squirrel')[0]
weasel = wn.synsets('weasel')[1] # index 0 is a sneaky person
squirrel.lowest_common_hypernyms(weasel)

snake = wn.synsets('snake')[0]
print(snake.lowest_common_hypernyms(weasel))
print(snake.lowest_common_hypernyms(squirrel))

turtle = wn.synsets('turtle')[1] # index 0 is a turtleneck (lmao)
print(turtle.lowest_common_hypernyms(snake))
print(turtle.lowest_common_hypernyms(squirrel))

fish = wn.synsets('fish', pos=wn.NOUN)[0] # pos = part of speech
swan = wn.synsets('swan')[0]
print(fish.lowest_common_hypernyms(turtle))
print(fish.lowest_common_hypernyms(snake))

louse = wn.synsets('louse')[0]
oyster = wn.synsets('oyster')[0]
print(louse.lowest_common_hypernyms(snake))
print(louse.lowest_common_hypernyms(oyster))

[Synset('vertebrate.n.01')]
[Synset('vertebrate.n.01')]
[Synset('reptile.n.01')]
[Synset('vertebrate.n.01')]
[Synset('vertebrate.n.01')]
[Synset('vertebrate.n.01')]
[Synset('animal.n.01')]
[Synset('invertebrate.n.01')]


In [12]:
class Node(object):

    def __init__(self, value, parent):
        self.value = value
        self.children = []
        if parent is not None:
            parent.children.append(self)

In [13]:
from print_tree import print_tree   #actually using package print-tree2 from pypi

class print_custom_tree(print_tree):

    def get_children(self, node):
        return node.children

    def get_node_str(self, node):
        return str(node.value)

In [15]:
animaln = Node('animal', None)
visited = []
skipped=0

for animals in simple_terms :
    if animals not in ['bombicum','og','verdaule','tellin','petit-gri','aucupio','daot','shell','mutton','barbel','pork']:   #not in wordnet or not in tree
        if animals == "turtledofe" :
            animals = "turtledove"
        elif animals in ["calendra",'calandra']:
            animals = 'lark'
        elif animals == 'water-dog':
            animals = 'waterdog'
        if animals in ['weasel','hog','turtle','cuckoo','codfish','mussel','chicken','goldfinch','turtledove','linnet']: #first meaning is not the animal. Birds tend to have another bird as first meaning
            ani = wn.synsets(animals)[1]
        elif animals in ['ewe','swallow','crayfish','partridge','hart','lark']:  #have to go to third meaning to get the animal
             ani = wn.synsets(animals)[2]
        elif animals in ['dragon']: #the mythical creature does not have a link to animal. This is "any of several small tropical Asian lizards capable of gliding by spreading winglike membranes on each side of the body"
            ani = wn.synsets(animals)[3]
            print(ani.definition())
        else :
            ani = wn.synsets(animals)[0]
        an = r_node(ani)
        lis = an.return_animal_path()[0]
        for i in range (1,len(lis)):
            name = lis[i].name().split('.')[0]
            if name in ['even-toed_ungulate','odd-toed_ungulate','orthopterous_insect','chordate','placental','ungulate','thrush','oscine','leporid','lagomorph','decapod_crustacean','chordate','diapsid','anapsid','chelonian','bovid','bovine',"anseriform_bird",'wading_bird','columbiform_bird','corvine_bird','cuculiform_bird','aquatic_vertebrate','bony_fish','teleost_fish','gadoid','ganoid','annelid','oligochaete','decapod','soft-finned_fish','phasianid','ambystomid','red_deer','hominid','agamid']: #- make everything go boom also things I don't care about
                skipped+=1
                visited.append(name)
            elif name not in visited :
                name_prec = lis[i-1-skipped].name().split('.')[0]
                skipped=0
                string = name+'n'+' = Node(name,'+ name_prec+'n)'
                exec(string)
                visited.append(name)
            else :
                skipped=0

print_custom_tree(animaln) 

any of several small tropical Asian lizards capable of gliding by spreading winglike membranes on each side of the body

                               ┌squirrel
                               ├porcupine
                        ┌rodent┼rat
                        │      ├mouse
                        │      └dormouse
                        │         ┌feline─cat
                        ├carnivore┼musteline_mammal─weasel
                        │         │      ┌wolf
                        │         └canine┤
                        │                └dog
                        ├swine─hog
                        │               ┌beef
                        │        ┌cattle┼cow
                        │        │      └ox
                        │        ├sheep─ewe
                        ├ruminant┤
                        │        ├deer─hart
                        │        └goat
                 ┌mammal┤
                 │      ├rabbit
                 │      │      ┌mule
             